In [ ]:
import statsmodels.api as sm
import linearmodels as lm
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Paths to the reduced data sets (containing 1,000 rows, allows us to work quicker before working on the full-fledged sample)
path_ana_red = 'copy_master_data/Anagrafica_ClubQ8_red.csv'
path_premi_red = 'copy_master_data/Premi_ClubQ8_red.csv'
path_rif_red = 'copy_master_data/Rifornimenti_Carburante_ClubQ8_red.csv'

path_ana = 'master_data/Anagrafica_ClubQ8.csv'
path_premi = 'master_data/Premi_ClubQ8.csv'
path_rif = 'master_data/Rifornimenti_Carburante_ClubQ8.csv'

df1 = pd.read_csv(path_ana, sep=';', decimal = ',')
df1.head() #OK, works as expected and converts the decimals

df2 = pd.read_csv(path_premi, sep=';', decimal = ',')
df3 = pd.read_csv(path_rif, sep=';', decimal = ',')

In [ ]:
#For reduced DB as there is an Unnamed column added when performing the function

#df1.drop(['Unnamed: 0'], axis = 1, inplace= True)
#df2.drop(['Unnamed: 0'], axis = 1, inplace= True)
#df3.drop(['Unnamed: 0'], axis = 1, inplace= True)

df1.head()

In [ ]:
df2.columns

In [ ]:
#df4 = pd.DataFrame(columns = 'Test')
df1['DATA_BATTESIMO'] = df1['DATA_BATTESIMO'].astype('datetime64')
df1['DATA_BATTESIMO'] = df1['DATA_BATTESIMO'].dt.date

df2['DATA_OPERAZIONE'] = df2['DATA_OPERAZIONE'].astype('datetime64')
df2['DATA_OPERAZIONE'] = df2['DATA_OPERAZIONE'].dt.date
df2.head()

In [ ]:
df3['DATA_OPERAZIONE'] = df3['DATA_OPERAZIONE'].astype('datetime64')
df3['DATA_OPERAZIONE'] = df3['DATA_OPERAZIONE'].dt.date
df3.head()

df2.sort_values(by=['DATA_OPERAZIONE'], inplace=True, ascending=False)
df2.drop_duplicates('COD_PAN_DA_POS', keep='first', inplace=True)

df4 = df2.merge(df3, how='inner', on='COD_PAN_DA_POS')
print(df4.columns)
df4 = df4.iloc[:,[0,1,9,12]]

cols = df4.columns.tolist()
new_cols = [cols[2], cols[3], cols[0], cols[1]]
df4 = df4[new_cols]
df4 = df4.rename(columns={'DATA_OPERAZIONE_y': 'Date of gas purchase', 'LITRI': 'Gas Purchase in liters', 'COD_PAN_DA_POS': 'Unique Customer Identifier', 'DATA_OPERAZIONE_x': 'Date of first prize claim'})

df4['Date difference'] = df4['Date of gas purchase'].sub(df4['Date of first prize claim'], axis = 0)
print(df4.dtypes)
df4.head()

In [ ]:
Y = df4.iloc[:,1].values.reshape(-1,1)
#df4['Date difference'] = df4['Date difference'].astype('float')  
df4['Date difference'] = df4['Date difference']  / np.timedelta64(1,'D')
print(df4.dtypes)
X = df4.iloc[:,4].values.reshape(-1,1)

#SkLearn LinearRegression - This will not be used here, we prefer OLS
#lr = LinearRegression()
#e = np.random.normal(size=len(df4))
#lr.fit(X,Y)
#Y_pred = lr.predict(X)


#StatsModels OLS
model = sm.OLS(Y, X)
results = model.fit()
print(results.summary())
print("Parameters: ", results.params)
print("R2: ", results.rsquared)

In [ ]:
pred_ols = results.get_prediction()

#iv_l not necessary here because you cannot fuel a negative quantity; iv_u consequently not needed neither
#iv_l = pred_ols.summary_frame()["obs_ci_lower"]
#iv_u = pred_ols.summary_frame()["obs_ci_upper"]

fig, ax = plt.subplots(figsize=(8, 6))

ax.plot(X, Y, "o", label="data")
ax.plot(X, results.fittedvalues, "r--.", label="OLS")
#ax.plot(X, iv_u, "r--")
#ax.plot(X, iv_l, "r--")
ax.legend(loc="best")
plt.title('Regression of the gas purchases on the date difference (refuelling - claiming first prize) - Full sample - No constant')
plt.xlabel('Date difference = Date of gas purchase - Date of claiming a first prize')
plt.ylabel('Gas purchase in liters')
plt.tight_layout()
plt.savefig('figures/regression-full-noconstant.png')
plt.show()

In [ ]:
#Using a random sample from our data : taking 1000 rows randomly

df4sample = df4.sample(1000)
Ysample = df4sample.iloc[:,1].values.reshape(-1,1)
Xsample = df4sample.iloc[:,4].values.reshape(-1,1)

model = sm.OLS(Ysample, Xsample)
results = model.fit()
print(results.summary())
print("Parameters: ", results.params)
print("R2: ", results.rsquared)

pred_ols = results.get_prediction()


fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(Xsample, Ysample, "o", label="data")
ax.plot(Xsample, results.fittedvalues, "r--.", label="OLS")
ax.legend(loc="best")
plt.title('Regression of the gas purchases on the date difference (refuelling - claiming first prize) - Sample 1000 - No constant')
plt.xlabel('Date difference = Date of gas purchase - Date of claiming a first prize')
plt.ylabel('Gas purchase in liters')
plt.tight_layout()
plt.savefig('figures/regression-sample-noconstant.png')
plt.show()

In [ ]:
#Using PanelOLS regressor to take into account time-fixed and entity-fixed effects

#Multi-index needed for PanelOLS
df4=df4.set_index(['Date of gas purchase','Unique Customer Identifier'])
print(df4.head())
Y = df4['Gas Purchase in liters']
X = df4['Date difference']
model = lm.PanelOLS(Y, X, entity_effects=True, time_effects=True).fit()
results = model.fit()
print(results.summary())
print("Parameters: ", results.params)
print("R2: ", results.rsquared)

In [ ]:
#path_rif_red = 'copy_master_data/Rifornimenti_Carburante_ClubQ8_red.csv'
#df5 = pd.read_csv(path_rif_red, sep=';', decimal = ',')
#low_liter = df5[df5.iloc[:,6] <= 100]
#low_liter.sort_values(by=['LITRI'])
print(df5sample.head())

df5sample = df4.sample(100000)
df5sample = df5sample[df5sample['Gas Purchase in liters'] <= 100]
Ysample = df5sample.iloc[:,0].values.reshape(-1,1)
Xsample = df5sample.iloc[:,2].values.reshape(-1,1)



model = sm.OLS(Ysample, Xsample)
results = model.fit()
print(results.summary())
print("Parameters: ", results.params)
print("R2: ", results.rsquared)

pred_ols = results.get_prediction()
#iv_l = pred_ols.summary_frame()["obs_ci_lower"]
#iv_u = pred_ols.summary_frame()["obs_ci_upper"]

fig, ax = plt.subplots(figsize=(8, 6))

ax.plot(Xsample, Ysample, "o", label="data")
ax.plot(Xsample, results.fittedvalues, "r--.", label="OLS")
#ax.plot(Xsample, iv_u, "r--")
#ax.plot(Xsample, iv_l, "r--")
ax.legend(loc="best")
plt.show()

In [ ]:
#sns.kdeplot(Xsample)
sns.displot(df5sample, x="Gas Purchase in liters", binwidth=3)
plt.show()